In [1]:
# Packages einlesen
import pandas as pd
import json
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Daten einlesen
data = pd.read_csv("/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Final Data/Utilitarian_Final.csv")


In [21]:
data = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Final Data/Utilitarian_Final.csv')
#data_hedonic_raw = pd.read_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Final Data/Hedonic_Final.csv')

# 1. Data Cleaning

In [3]:
# Deleting special characters and review with less than 3 words
data.dropna(subset=['text'], inplace=True)

# Deleting empty rows
data = data [data['text']!= ' ']

In [4]:
# Deleting the string "[[Video ID:...]]" from the review text
import re
# Initialize an empty list to store VideoID and numbers
video_id_and_number_list = []

# Define the regular expression pattern to match the [[VIDEOID:...]] text and extract the VideoID and numbers
video_id_and_number_pattern = r'\[\[VIDEOID:([^\]]*)\]\]'

# Extract VideoID and numbers from each row in the 'text' column and save them into the list
for text in data['text']:
    matches = re.findall(video_id_and_number_pattern, text)
    for match in matches:
        video_id_and_number_list.append('[[VIDEOID:' + match +"]]")

# Use list to iterate through the 'text' column and replace the VideoID and numbers with an empty string
for video_id_and_number in video_id_and_number_list:
    data['text'] = data['text'].str.replace(video_id_and_number, '')



In [5]:
import re

# Define the regular expression pattern to match the entire [[VIDEOID:...]] text and any following numbers
video_id_and_number_pattern = r'\[\[VIDEOID:([0-9]+)'

# Use regular expressions to remove the string "[[VIDEOID:...]]" and any following numbers from the review text
data['text'] = data['text'].str.replace(video_id_and_number_pattern, '')


/var/folders/ss/4y6dgpw950348n7xdxy73vnm0000gn/T/ipykernel_7686/2937091207.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text'] = data['text'].str.replace(video_id_and_number_pattern, '')


In [6]:
import re

# Define the regular expression pattern to match the entire [[VIDEOID:...]] text and any following alphanumeric characters
video_id_and_number_pattern = r'\[\[VIDEOID:([^\s]+)'

# Use regular expressions to remove the string "[[VIDEOID:...]]" and any following alphanumeric characters from the review text
data['text_cleaned'] = data['text_cleaned'].str.replace(video_id_and_number_pattern, '')


/var/folders/ss/4y6dgpw950348n7xdxy73vnm0000gn/T/ipykernel_7686/4226586295.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  data['text_cleaned'] = data['text_cleaned'].str.replace(video_id_and_number_pattern, '')


## 2. Text Preprocessing



In [7]:
import spacy

# Load the English language model
nlp = spacy.load("en_core_web_sm")

# Function to preprocess text using spaCy with lemmatization and lowercasing

def preprocess_text(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Lemmatization and lowercasing
    processed_text = ' '.join([token.lemma_.lower() for token in doc])
    
    return processed_text


In [8]:
# Adding another column, that filters out stop words and punctuation/symbols
import string

# Define a set of stop words
stop_words = spacy.lang.en.stop_words.STOP_WORDS

# Function to preprocess text using spaCy
def preprocess_text_stops(text):
    # Process the text with spaCy
    doc = nlp(text)
    
    # Lemmatization, lowercasing, and removal of symbols, punctuation, and stop words
    processed_text = ' '.join([token.lemma_.lower() for token in doc if token.is_alpha and not token.is_stop and not token.is_punct])
    
    return processed_text

In [9]:
# Apply the preprocessing function to the "text" column
data['text_cleaned'] = data['text'].apply(preprocess_text)

# Apply the preprocessing function with stopword removal to the "text" column
data['text_cleaned1'] = data['text'].apply(preprocess_text_stops)


## Using Sentiment Analysis for Review Text

In [5]:
# Download the pre-trained BERT model and tokenizer
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [20]:
# Load the pre-trained BERT model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("LiYuan/amazon-review-sentiment-analysis")
sentiment_model = AutoModelForSequenceClassification.from_pretrained("LiYuan/amazon-review-sentiment-analysis")


In [21]:
# Build Function to analyze the sentiment of a text

def analyze_sentiment(text):
    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    # Perform sentiment analysis
    outputs = sentiment_model(**inputs)
    # Get the predicted label
    predicted_label = torch.argmax(outputs.logits)
    return predicted_label.item()  # Return the predicted label as an integer


In [22]:
# Apply the function to the utilitarian product
data['sentiment'] = data['text'].apply(analyze_sentiment)


In [4]:

# Define the mapping dictionary for transforming sentiment score to match rating score 1-5, so that 0-4 gets 1-5
sentiment_mapping = {0: 1, 1: 2, 2: 3, 3: 4, 4: 5}

# Apply the mapping to the 'sentiment' column
data['sentiment'] = data['sentiment'].replace(sentiment_mapping)

# Verify the transformation
print(data['sentiment'])


0        5
1        5
2        5
3        3
4        5
        ..
22780    5
22781    1
22782    1
22783    5
22784    5
Name: sentiment, Length: 22785, dtype: int64


## Trying for an different Sentiment Model (Which classifies as neutral, negative, positive)

In [7]:
data = data[data['helpful_vote'] != 0]
data

In [22]:
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

# Load the sentiment analysis model and tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

def analyze_sentiment(text):
    # Tokenize the text into chunks of tokens with a max length of 511
    tokens = tokenizer.tokenize(text)
    chunk_size = 511
    chunks = [tokens[i:i + chunk_size] for i in range(0, len(tokens), chunk_size)]
    
    sentiment_scores = []
    
    for chunk in chunks:
        # Convert tokens back to text
        chunk_text = tokenizer.convert_tokens_to_string(chunk)
        # Tokenize the chunk text
        inputs = tokenizer(chunk_text, return_tensors='pt', padding=True, truncation=True, max_length=chunk_size)
        # Perform sentiment analysis
        outputs = model(**inputs)
        scores = softmax(outputs.logits[0].detach().numpy())
        predicted_label_id = np.argmax(scores)
        predicted_label = config.id2label[predicted_label_id]
        sentiment_scores.append(predicted_label)
    
    # Aggregate the sentiment scores (e.g., by taking the majority vote)
    return majority_vote(sentiment_scores)

def majority_vote(sentiment_scores):
    # Count the occurrences of each sentiment label
    label_counts = {label: sentiment_scores.count(label) for label in set(sentiment_scores)}
    # Find the label with the highest count
    majority_label = max(label_counts, key=label_counts.get)
    return majority_label

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

# Assuming df['text_cleaned'] contains the cleaned review texts
data['Sentiment_Classification'] = data['text_cleaned'].apply(analyze_sentiment)

# Display the results
print(data[['text_cleaned', 'Sentiment_Classification']].head())


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


                                        text_cleaned Sentiment_Classification
0  i use more experience roll but this be great f...                 positive
1  i expect just to have some extra roll on hand ...                 positive
2  my price line for find deal on toilet paper be...                 positive
3  container be filthy and have huge gap expose c...                 negative
4                                  quality and price                  neutral


In [23]:
data.to_csv('/Users/paulahofmann/Documents/Coding/Online-Review/FeaturePreperation/Data_with_Features/Final Data/Utilitarian_Final.csv', index=False)